# Introdução

Esse Notebook será responsável pelo preprocessamento dos dados contidos em ./data/raw para formatos compatíveis e otimizados para o treinamento de cada modelo
As definições dos parâmetros do treinamento aos quais os modelos deverão solucionar já foram definidas no notebook "Coleta de Dados"

Modelos a serem Criados:

1. Modelo Linear: MLP sem funções de ativação, composta apenas de somas lineares
2. MLP: rede neural - efetivamente identica ao modelo linear, no entanto, apresenta função de ativação ao final do somatório de funções lineares
3. LSTM: Um modelo de rede neural recorrente, com capacidade de diferenciação de informação de curto e longo prazo
4. TFT: Modelo baseado em LLMs desenvolvido pela microsoft - servirá como um comparativo mais moderno


## Dependências

In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"
# Dependências mínimas para TFT — simples e com foco em GPU
# try:
#     import torch
# except Exception:
#     import shutil
#     has_gpu = shutil.which("nvidia-smi") is not None
#     if has_gpu:
#         # tenta instalar com suporte CUDA (ajuste a versão cu de acordo com sua stack, p.ex. cu121)
#         %pip install -q torch --index-url https://download.pytorch.org/whl/cu121
#     else:
#         %pip install -q torch --index-url https://download.pytorch.org/whl/cpu
#     # libs do pipeline TFT
#     %pip install -q pytorch-lightning pytorch-forecasting

#     import torch



# print(f"torch={torch.__version__} | cuda={torch.cuda.is_available()}")

print("Verificando dependências (pyarrow para Parquet)...")

try:
    import pyarrow as pa
    print(f"PyArrow disponível: {pa.__version__}")
except Exception:
    print("Instalando pyarrow...")
    !pip install --upgrade "pyarrow>=18" --quiet
    import importlib
    importlib.invalidate_caches()
    import pyarrow as pa
    print(f"PyArrow instalado: {pa.__version__}")

# fastparquet é opcional
try:
    import fastparquet  # noqa: F401
    print("fastparquet disponível (opcional)")
except Exception:
    pass

# Outras bibliotecas sob demanda
for lib in [
    "numpy", "python-dotenv", "pandas", "matplotlib", "seaborn",
    "scikit-learn", "tensorflow[and-cuda]", "keras", "lxml", "pytz"
]:
    try:
        __import__(lib)
    except ImportError:
        print(f"Instalando {lib}...")
        !pip install {lib} --quiet

print("Dependências prontas")

Verificando dependências (pyarrow para Parquet)...
PyArrow disponível: 22.0.0
Instalando python-dotenv...
Instalando scikit-learn...
Instalando tensorflow[and-cuda]...
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.1.4 which is incompatible.
torch 2.5.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.9.79 which is incompatible.
torch 2.5.1+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.9.86 which is incompatible.
torch 2.5.1+cu121 requires nvidia-cuda-runtime-cu12==12.1.105; platform_system == "Linux" and pla

## VARIÁVEIS NECESSÁRIAS

In [2]:
# Imports centralizados
import os, json, time, gc, concurrent.futures, datetime
from typing import Any, Dict, List, Optional, Tuple
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor, as_completed
import multiprocessing

# TensorFlow / Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, mixed_precision
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping as TFEarlyStopping, ReduceLROnPlateau as TFReduceLROnPlateau

# PyTorch / Lightning / TFT
import torch
from lightning.pytorch import Trainer, seed_everything
from lightning.pytorch.callbacks import EarlyStopping as LGEarlyStopping, LearningRateMonitor as LGLearningRateMonitor, ModelCheckpoint as LGModelCheckpoint
from pytorch_forecasting.models import TemporalFusionTransformer
from pytorch_forecasting.metrics import QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

# Parquet
import pyarrow as pa
import pyarrow.parquet as pq

# Local preprocessor classes
from preprocessor import LinearPreprocessor, LSTMPreprocessor, TFTPreprocessor

# Silenciando Warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=Warning)  # last resort
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)



# ==============================================
# GPU CONFIGURATION
# ==============================================
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    mixed_precision.set_global_policy('mixed_float16')
    print(f"✅ GPU detected ({gpus[0].name}) - using mixed precision.")
else:
    print("⚠️ No GPU detected, running on CPU.")

# Carregar variáveis de ambiente do .env
load_dotenv()
# ---------------- CONFIG ---------------- #
COUNTRY_DOMAINS = {
    "FR": {"domain": "10YFR-RTE------C"},
    "ES": {"domain": "10YES-REE------0"},
    "PT": {"domain": "10YPT-REN------W"}
}

DATA_ITEMS = [
    {'key': 'load_total', 'documentType': 'A65', 'processType': 'A16', 'domainParam': 'outBiddingZone_Domain', 'parser': 'load'},
    {'key': 'market_prices', 'documentType': 'A44', 'processType': 'A07', 'domainParamIn': 'in_Domain', 'domainParamOut': 'out_Domain', 'parser': 'price'}
]

ENTSOE_TOKEN = os.environ.get("ENTSOE_SECURITY_TOKEN")
BASE_URL = "https://web-api.tp.entsoe.eu/api"
MAX_WORKERS = 100
RAW_DIR = os.path.join("data", "raw")
PARQUET_COMPRESSION = "zstd"
os.makedirs(RAW_DIR, exist_ok=True)

# ==============================================
# DICIONÁRIO DE Treinamento
# ==============================================
treinamento = {
    "name": "treinamento",
    "data_dir": "data/treinamento",
    "feats": ["country", "datetime", "quantity_MW"],
    "tgts": ["quantity_MW"],
    "vals": ["quantity_MW"],
    "lag": 10 * 24,
    "lead": 3 * 24,
    "seq_len": 10 * 24,
    "countries": list(COUNTRY_DOMAINS),
    "noise": False,
    "train": True
}

perguntas = [
    {
        "name": "N1A",
        "data_dir": "data/N1A",
        "feats": ["country", "datetime", "quantity_MW"],
        "tgts": ["quantity_MW"],
        "vals": ["quantity_MW"],
        "lag": 10 * 24,
        "lead": 3 * 24,
        "seq_len": 3 * 24,
        "countries": ["ES"],
        "noise": False
    },
    {
        "name": "N1B",
        "data_dir": "data/N1B",
        "feats": ["country", "datetime", "quantity_MW"],
        "tgts": ["quantity_MW"],
        "vals": ["quantity_MW"],
        "lag": 10 * 24,
        "lead": 3 * 24,
        "seq_len": 7 * 24,
        "countries": ["ES"],
        "noise": False
    },
    {
        "name": "N1C",
        "data_dir": "data/N1C",
        "feats": ["country", "datetime", "quantity_MW"],
        "tgts": ["quantity_MW"],
        "vals": ["quantity_MW"],
        "lag": 10 * 24,
        "lead": 3 * 24,
        "seq_len": 10 * 24,
        "countries": ["ES"],
        "noise": False
    },
    {
        "name": "N2A",
        "data_dir": "data/N2A",
        "feats": ["country", "datetime", "quantity_MW"],
        "tgts": ["quantity_MW"],
        "vals": ["quantity_MW"],
        "lag": 10 * 24,
        "lead": 3 * 24,
        "seq_len": 3 * 24,
        "countries": COUNTRY_DOMAINS.keys(),
        "noise": False
    },
    {
        "name": "N2B",
        "data_dir": "data/N2B",
        "feats": ["country", "datetime", "quantity_MW"],
        "tgts": ["quantity_MW"],
        "vals": ["quantity_MW"],
        "lag": 10 * 24,
        "lead": 3 * 24,
        "seq_len": 7 * 24,
        "countries": COUNTRY_DOMAINS.keys(),
        "noise": False
    },
    {
        "name": "N2C",
        "data_dir": "data/N2C",
        "feats": ["country", "datetime", "quantity_MW"],
        "tgts": ["quantity_MW"],
        "vals": ["quantity_MW"],
        "lag": 10 * 24,
        "lead": 3 * 24,
        "seq_len": 10 * 24,
        "countries": COUNTRY_DOMAINS.keys(),
        "noise": False
    }
]



/home/victor-bertini/Documentos/tcc_2025/TCC-2025/tfc_venv/lib/python3.12/site-packages/pytorch_forecasting/models/base/_base_model.py:28: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


✅ GPU detected (/physical_device:GPU:0) - using mixed precision.


# Capítulo 1: Préprocessamento de dados

Etapa de contrução da pipelines de pre-processamento de dados


## Preprocessamento do Modelo Linear

Esse modelo deve será contruido a partir de lags e leads passados como parâmetros na função, resultando na contrução de novas colunas lead lag, assim gerando uma flat matrix 2D que será usada no modelo linear

Observação importante: lag e lead são inteiros e representam o máximo de passos; o pipeline expande para intervalos 1..N automaticamente. Por exemplo, lag=96 gera features com defasagens de 1 a 96; lead=96 gera alvos de 1 a 96.

Os arquivos do modelo serão salvos em Parquet, já que o modelo linear será construído usando TensorFlow (carregado via Parquet→NumPy→tf.data)

No caso o Preprocessador do modelo linear será igual ao pré-processador do MLP 

## Preprocessamento do Modelo LSTM

O preprocessador do LSTM deve ser capaz de gerar tensores de dimensão 3, no seguinte formato (n_batch, seq_len, features) e (n_batch, seq_len, features)  para alimentação do modelo e validação das métricas do modelo

Os arquivos do modelo serão salvos em Parquet (colunas X e Y como listas fixas), e carregados via Parquet→NumPy→tf.data

## Preprocessamento do Modelo TFT (PyTorch)

O preprocessador do LSTM deve ser capaz de gerar tensores de dimensão 3, no seguinte formato (n_batch, seq_len, features) e (n_batch, seq_len, features)  para alimentação do modelo e validação das métricas do modelo

Os artefatos de dados serão salvos em Parquet (Keras e TFT).

# Capítulo 2 — Construção dos Modelos

A seguir, definimos construtores simples e eficientes para cada modelo (Linear, LSTM, TFT e TimesFM),
prontos para uso em rotinas de otimização de hiperparâmetros (por exemplo, Optuna). Cada construtor
recebe um dicionário de parâmetros (`params`) e retorna um modelo compilado.

## Construção do Modelo Linear/MLP

Objetivo: Criar um regressor simples (MLP), com capacidade de redução para um modelo apenas lienar - pela exclusão da camada de ativação - para prever `target_cols` a partir de `feature_cols`.

Contrato rápido:
- Entrada: vetor de tamanho `x_dim` (número de features)
- Saída: vetor de tamanho `y_dim` (número de targets)
- Parâmetros (exemplos): hidden_units, activation, dropout, lr, l2

In [3]:
# Linear/MLP model + loaders (Parquet only)
# (imports centralizados na célula 5)
from tensorflow import keras
from tensorflow.keras import layers, regularizers

def build_linear_model(x_dim: int, y_dim: int, params: dict, linear: bool = False) -> keras.Model:
    """Builds either a pure linear or a simple MLP model for regression."""

    lr = params.get("lr", 1e-3)
    l2 = params.get("l2", 0.0)
    dropout = params.get("dropout", 0.0)
    hidden_units = params.get("hidden_units", [128, 64])
    activation = params.get("activation", "relu")

    inputs = keras.Input(shape=(x_dim,), name="features")

    if linear:
        x = inputs
    else:
        x = inputs
        for i, units in enumerate(hidden_units):
            x = layers.Dense(units, activation=activation,
                             kernel_regularizer=regularizers.l2(l2))(x)
            if dropout > 0:
                x = layers.Dropout(dropout)(x)

    outputs = layers.Dense(y_dim, activation=None, name="targets")(x)

    model = keras.Model(inputs, outputs, name="linear_model" if linear else "mlp_model")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss="mse",
        metrics=["mae"]
    )
    return model


def build_mlp_model(x_dim: int, y_dim: int, params: Dict[str, Any]) -> keras.Model:

    # --- HYPERPARAMS ---
    hidden_units = params.get('hidden_units', [128, 64])  
    dropout = float(params.get('dropout', 0.0))            
    lr = float(params.get('lr', 1e-3))                     
    l2 = float(params.get('l2', 1e-6))                     
    activation = params.get('act', 'relu')

    # --- MODEL ---
    inputs = keras.Input(shape=(x_dim,), name='features')
    x = inputs  # 🚫 no LayerNormalization()

    # hidden layers
    for i, units in enumerate(hidden_units):
        x = layers.Dense(
            units,
            activation=activation,
            kernel_initializer='he_normal',
            kernel_regularizer=keras.regularizers.l2(l2),
            name=f"dense_{i}"
        )(x)

        if dropout > 0:
            x = layers.Dropout(dropout, name=f"drop_{i}")(x)

    # output layer
    outputs = layers.Dense(y_dim, activation=None, name='targets')(x)

    model = keras.Model(inputs, outputs, name='mlp_relu')

    # --- OPTIMIZER ---
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss='mse',
        metrics=['mae']
    )

    return model


## Construção do Modelo LSTM

Objetivo: um regressor denso simples (MLP) para prever `target_cols` a partir de `feature_cols`.

Contrato rápido:
- Entrada: vetor de tamanho `x_dim` (número de features)
- Saída: vetor de tamanho `y_dim` (número de targets)
- Parâmetros (exemplos): hidden_units, activation, dropout, lr, l2

In [28]:
def build_lstm_model(
    seq_len: Optional[int],
    x_dim_num: int,
    y_dim: int,
    lead: int,
    num_countries: int,
    params: Dict[str, Any],
) -> keras.Model:
    """
    LSTM seq2seq com embedding categórico ('country') e suporte a seq_len variável.

    Estrutura:
      [num_feats_input] + [country_id_input -> Embedding -> RepeatVector(seq_len)]
      ↓ concatenação
      LSTM Encoder → RepeatVector(lead) → LSTM Decoder → TimeDistributed(Dense(y_dim))
    """

    from tensorflow import keras
    from tensorflow.keras import layers
    import tensorflow as tf

    # --- HYPERPARAMS ---
    lu = params.get("lstm_units", 128)
    if isinstance(lu, (list, tuple)):
        enc_units = int(lu[0])
        dec_units = int(lu[-1])
    else:
        enc_units = dec_units = int(lu)

    dropout = float(params.get("dropout", 0.1))
    rdrop = float(params.get("recurrent_dropout", params.get("rec_dropout", 0.0)))
    lr = float(params.get("lr", 1e-3))
    bidir = bool(params.get("bidirectional", False))
    emb_dim = int(params.get("emb_dim", 8))

    # --- INPUTS ---
    num_in = keras.Input(shape=(None, x_dim_num), name="num_feats")
    country_in = keras.Input(shape=(1,), dtype="int32", name="country_id")

    # --- EMBEDDING ---
    emb = layers.Embedding(
        input_dim=num_countries,
        output_dim=emb_dim,
        name="country_emb"
    )(country_in)

    # ✅ Simple, symbolic-safe repeat
    emb = layers.Reshape((-1,))(emb)  # from (None, 1, emb_dim) → (None, emb_dim)
    emb = layers.RepeatVector(seq_len, name="repeat_country_emb")(emb)
    x = layers.Concatenate(axis=-1, name="concat_inputs")([num_in, emb])

    # --- ENCODER ---
    if bidir:
        enc = layers.Bidirectional(
            layers.LSTM(enc_units, dropout=dropout, recurrent_dropout=rdrop, return_sequences=False),
            name="enc_bi"
        )(x)
    else:
        enc = layers.LSTM(enc_units, dropout=dropout, recurrent_dropout=rdrop, return_sequences=False, name="enc")(x)

    # --- DECODER ---
    rep = layers.RepeatVector(lead, name="repeat_lead")(enc)
    dec = layers.LSTM(dec_units, dropout=dropout, recurrent_dropout=rdrop, return_sequences=True, name="dec")(rep)

    # --- PROJECTION HEAD ---
    proj = layers.TimeDistributed(
        layers.Dense(64, activation="relu", kernel_initializer="he_normal"),
        name="td_dense"
    )(dec)
    out = layers.TimeDistributed(layers.Dense(y_dim), name="td_out")(proj)

    # --- COMPILE ---
    model = keras.Model(inputs=[num_in, country_in], outputs=out, name="lstm_seq2seq_emb")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr, clipnorm=1.0),
        loss="mse",
        metrics=["mae"]
    )

    return model


## Construção do Modelo TFT (Temporal Fusion Transformer)

**Objetivo:** prever `target_cols` a partir de `feature_cols` utilizando a implementação oficial `keras_tft`,  
que integra **seleção de variáveis dinâmicas**, **blocos LSTM**, **atenção temporal multi-cabeças** e **gating residual** em um único modelo interpretable.

**Contrato rápido:**
- **Entrada:** `tf.data.Dataset` com tensores no formato `(batch, seq_len, x_dim)`  
- **Saída:** tensor contínuo de tamanho `y_dim` *(ou `dec_len × y_dim` para horizontes múltiplos)*

**Parâmetros (exemplos):**  
`hidden_size` (tamanho interno das camadas GRN) · `lstm_layers` · `num_heads` (atenção) · `dropout` · `learning_rate` · `output_size` · `seq_len`

**Componentes internos (`keras_tft`):**  
Variable Selection Network → LSTM Encoder/Decoder → Multi-Head Temporal Attention → Gated Residual Network → Camada de projeção final

**Compatibilidade:**  
Totalmente compatível com o pipeline atual em Parquet do LSTM, recebendo o mesmo formato de dados  
(`(batch, seq_len, features)`), permitindo substituição direta do modelo sem alterar o pré-processamento.



In [5]:
# TFT model builder (imports centralizados na célula 5)

def build_tft_model(
    params: Dict[str, Any]
):
    """
    Constrói um Temporal Fusion Transformer (TFT) com parâmetros configuráveis.

    Args:
        x_dim: número de features de entrada
        y_dim: número de targets
        seq_len: tamanho da sequência temporal
        params: dicionário de hiperparâmetros (hidden_size, dropout, lstm_layers, etc.)
        max_encoder_length: tamanho da janela passada (encoder)
        max_prediction_length: tamanho do horizonte de previsão (decoder)
    """

    hidden_size = int(params.get("hidden_size", 64))
    dropout = float(params.get("dropout", 0.1))
    lstm_layers = int(params.get("lstm_layers", 1))
    attention_head_size = int(params.get("num_heads", 4))
    lr = float(params.get("lr", 1e-3))

    model = TemporalFusionTransformer.from_dataset(
        params["dataset"],  # dataset preparado via TimeSeriesDataSet
        learning_rate=lr,
        hidden_size=hidden_size,
        dropout=dropout,
        lstm_layers=lstm_layers,
        attention_head_size=attention_head_size,
        loss=QuantileLoss([0.5]),
        log_interval=10,
        log_val_interval=1
    )

    return model

# Capítulo 3 - Contrução da Pipelines de dados dos modelos

A função de pipeline organiza o fluxo de dados para, de forma mais concisa e organizada, treinar o modelo, sendo capaz de mostrar a progressão das perdas a medida que as épocas de treinamento passam - Esse display esté disponível no notebook "Resultados"

O resultado da pipeline é um gráfico com a evolução de todas as métricas e o salvamento do modelo treinado dentro da pasta ./modelo/{Nome_Problema}/{Nome_Modelo}

Assim podendo ser facilmente reutilizado futuramente para um notebook comparativo

In [ ]:
# Utilitário para salvar modelos Keras (imports centralizados na célula 5)

def save_model(model, path: str, format: str | None = None, include_optimizer: bool = True) -> str:
    """
    Salva um modelo Keras em formato padronizado.

    Regras:
    - Se `path` terminar com .keras ou .h5, salva exatamente nesse arquivo.
    - Se `format == 'savedmodel'`, salva no diretório indicado (SavedModel).
    - Caso contrário, adiciona sufixo .keras a `path` (arquivo único Keras v3).

    Retorna o caminho final salvo (arquivo ou diretório) e grava um meta.json ao lado.
    """
    # Inferência de formato por extensão
    ext = None
    lower = path.lower()
    if lower.endswith(".keras"):
        ext = "keras"
    elif lower.endswith(".h5") or lower.endswith(".hdf5"):
        ext = "h5"

    # Normalização de destino
    if format == "savedmodel":
        # Diretório SavedModel
        save_dir = path
        os.makedirs(save_dir, exist_ok=True)
        model.save(save_dir, include_optimizer=include_optimizer)
        meta_path = os.path.join(save_dir, "model.meta.json")
        final_path = save_dir
    else:
        if ext is None:
            # Força arquivo .keras por padrão
            path = f"{path}.keras"
            ext = "keras"
        # Cria diretório pai
        parent = os.path.dirname(path) or "."
        os.makedirs(parent, exist_ok=True)
        # Salva arquivo único
        model.save(path, include_optimizer=include_optimizer)
        meta_path = f"{path}.meta.json"
        final_path = path

    # Meta básico ao lado do artefato
    try:
        meta = {
            "saved_at": datetime.datetime.utcnow().isoformat() + "Z",
            "keras_version": getattr(model, "keras_version", None),
            "model_name": getattr(model, "name", None),
            "trainable_params": int(getattr(model, "count_params", lambda: 0)()),
            "format": "savedmodel" if format == "savedmodel" else ext,
        }
        with open(meta_path, "w", encoding="utf-8") as f:
            json.dump(meta, f, ensure_ascii=False, indent=2)
    except Exception as e:
        print(f"[WARN] Falha ao escrever meta.json: {e}")

    print(f"[💾] Modelo salvo em: {final_path}")
    return final_path

## Pipeline dos Modelos Lineares

Pipeline de preprocessamento e de treinamento

In [7]:
# Pipelines dos Modelos Lineares (imports centralizados na célula 5)

def linear_preproccess_pipeline(
    preproc: LinearPreprocessor,
    destino_dir: str,
    value_cols: List[str],
    save_instance: bool = True,
    seq_len=None,
    masked_value=None,
    train: bool = False,
) -> Tuple[LinearPreprocessor, Dict[str, Any]]:
    """
    Pipeline completa de pré-processamento (Parquet only) e treinamento de 3 modelos
    usando apenas a classe unificada Preprocessor.

    Retorna o preprocessor (para reutilizar escalers/mapeamentos) e um dicionário
    com resultados/resumos do treinamento dos modelos lineares/MLP.
    """


    print("🔄 Carregando dados brutos ...")
    preproc.load_data()

    print("🔤 Encoding de colunas categóricas/temporais ...")
    preproc.encode(encode_cols='datetime', encode_method='time_cycle', train=train)
    preproc.encode(encode_cols='country', encode_method='label', train=train)

    print("✂️ Split train/val/test ...")
    preproc.split_train_val_test(train_size=0.8, val_size=0.1, test_size=0.1, time_col='datetime')

    print("🔳 Construindo matrizes planas para Linear/MLP ...")
    preproc.build_flat_matrices_splits(
        value_cols=value_cols,
        dropna=True,
        group_cols=['country'],
        time_col='datetime',
        seq_len=seq_len,
        mask_value=masked_value

    )

    print("📐 Normalização ...")
    new_value_cols = []
    new_value_cols.append(preproc.feature_cols)
    new_value_cols.append(preproc.target_cols)
    substring = value_cols[0]  # palavra a procurar
    filtered_cols = [
        c for sublist in new_value_cols for c in sublist if substring in c
    ]
    preproc.normalize_splits(value_cols=filtered_cols, normalization_method='minmax', train=train)



    print("💾 Salvando parquets LINEAR/MLP ...")
    preproc.save_linear_splits_parquet(basename='linear_dataset')

    if save_instance and train:
       path = os.path.join(destino_dir, "preprocessor")
       preproc.save_instance(path, name="linear_preproc.pkl")

    print("📦 Dataset Parquet carregado para treinamento.")

    return preproc

def linear_train_pipeline(
    problem_name: str,
    data_dir: str,
    batch_size: int = 128,
    configs: Dict[str, Dict[str, Any]] = None
):
    # ----------------------------
    # Parquet datasets
    # ----------------------------
    dataset_train, meta_tr = LinearPreprocessor.load_linear_parquet_dataset(data_dir=data_dir, split='train', batch_size=batch_size, shuffle=True)
    dataset_val, meta_va = LinearPreprocessor.load_linear_parquet_dataset(data_dir=data_dir, split='val', batch_size=batch_size, shuffle=False)


    x_dim = int(meta_tr["x_dim"])
    y_dim = int(meta_tr["y_dim"])
    print("📦 Dataset Parquet carregado para treinamento.")

    histories = {}
    models = {}

    early_stopping = TFEarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True, verbose=0)
    reduce_lr = TFReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=6, min_lr=1e-6, verbose=0)

    # ----------------------------
    # Treinamento de cada modelo
    # ----------------------------
    for name, params in configs.items():
        print(f"\n🚀 Treinando modelo {name}...")

        linear_bool = params.get("linear")
        if linear_bool:
            model = build_linear_model(x_dim=x_dim, y_dim=y_dim, params=params, linear=linear_bool)
        else:
            model = build_mlp_model(x_dim=x_dim, y_dim=y_dim, params=params)
        

        hist = model.fit(
            dataset_train,
            validation_data=dataset_val,
            epochs=200,
            callbacks=[early_stopping, reduce_lr],
            verbose=2
)

        histories[name] = hist
        models[name] = model
        print(f"✅ {name} concluído - Val Loss: {min(hist.history['val_loss']):.6f}")

    # ----------------------------
    # Salvando modelos
    # ----------------------------
    for name, model in models.items():
        # Salvando modelo no path /modelos/{nome do problema}/{nome do modelo}
        save_model(model, path = f"./modelos/{problem_name}/{name}")
    
    
    return models

## Pipelines dos modelos LSTM
Implementação e uso dos preprocessors e treinadores LSTM para séries temporais (janelas seq_len e lead).

In [8]:
import tensorflow as tf
import numpy as np

def sanity_check_prefetch(ds: tf.data.Dataset, n: int = 3, n_vals: int = 5):
    """
    Quick sanity check for tf.data.Dataset (prefetched/batched).
    Prints shape, dtype, and sample values for a few batches.

    Args:
        ds : tf.data.Dataset — dataset to inspect
        n  : int — number of batches to preview
        n_vals : int — number of feature values to display (first & last)
    """
    print("🔍 Dataset sanity check...")
    for i, (x, y) in enumerate(ds.take(n)):
        print(f"\n[Batch {i+1}] ===============================")

        # --- X INFO ---
        if isinstance(x, dict):
            print("X: (dict)")
            for k, v in x.items():
                arr = v.numpy() if tf.is_tensor(v) else np.array(v)
                print(f" • {k:<20} shape={arr.shape} dtype={arr.dtype}")
                flat = arr.flatten()
                if flat.size > 0:
                    head = np.array2string(flat[:n_vals], precision=4, separator=", ")
                    tail = np.array2string(flat[-n_vals:], precision=4, separator=", ")
                    print(f"   first: {head}")
                    print(f"   last : {tail}")
        else:
            arr = x.numpy() if tf.is_tensor(x) else np.array(x)
            print(f"X shape={arr.shape} dtype={arr.dtype}")
            flat = arr.flatten()
            if flat.size > 0:
                head = np.array2string(flat[:n_vals], precision=4, separator=", ")
                tail = np.array2string(flat[-n_vals:], precision=4, separator=", ")
                print(f"  first: {head}")
                print(f"  last : {tail}")

        # --- Y INFO ---
        arr_y = y.numpy() if tf.is_tensor(y) else np.array(y)
        print(f"Y shape={arr_y.shape} dtype={arr_y.dtype}")
        flat_y = arr_y.flatten()
        if flat_y.size > 0:
            head_y = np.array2string(flat_y[:n_vals], precision=4, separator=", ")
            tail_y = np.array2string(flat_y[-n_vals:], precision=4, separator=", ")
            print(f"  y first: {head_y}")
            print(f"  y last : {tail_y}")

        if i + 1 >= n:
            break
    print("\n✅ Done.")


In [19]:
# Pipelines dos modelos LSTM (imports centralizados na célula 5)

def lstm_preproccess_pipeline(
    preproc: LSTMPreprocessor,
    destino_dir: str,
    value_cols: List[str],
    save_instance: bool = True,
    train: bool = False
) -> Tuple[LSTMPreprocessor, Dict[str, Any]]:
    

    print("🔄 Carregando dados brutos ...")
    preproc.load_data()
    
    print("🔤 Encoding ...")
    preproc.encode(encode_cols="datetime", encode_method="time_cycle", train=train)
    preproc.encode(encode_cols="country", encode_method="label", train=train)

    print("✂️ Split train/val/test ...")
    preproc.split_train_val_test(train_size=0.6, val_size=0.2, test_size=0.2, time_col="datetime")
    
    print("📐 Normalização ...")
    preproc.normalize_splits(value_cols=value_cols, normalization_method="standard", train=train)

    print("🔳 Construindo matrizes sequenciais para LSTM ...")
    preproc.build_sequence_matrix_splits(
        group_cols=['country'],
        time_col="datetime"
    )

    print("💾 Salvando parquets LSTM ...")
    preproc.save_splits_parquet(basename="lstm_dataset")

    if save_instance and train:
        path = os.path.join(destino_dir, "preprocessor")
        preproc.save_instance(path, name="lstm_preproc.pkl")

    print("📦 Dataset LSTM Parquet carregado para treinamento.")
    return preproc



def lstm_train_pipeline(
    problem_name: str,
    data_dir: str,
    seq_len: int,
    batch_size: int = 128,
    configs: Dict[str, Dict[str, Any]] = None
) -> Dict[str, keras.Model]:
    # ----------------------------
    # Parquet datasets
    # ----------------------------
    dataset_train, meta_tr = LSTMPreprocessor.load_lstm_parquet_dataset(data_dir=data_dir, split="train", batch_size=batch_size, shuffle=False)
    dataset_val, meta_va = LSTMPreprocessor.load_lstm_parquet_dataset(data_dir=data_dir, split="val", batch_size=batch_size, shuffle=False)

    x_dim = int(meta_tr["x_dim"]) ; y_dim = int(meta_tr["y_dim"]) ; seq_len_m = int(meta_tr["seq_len"]) ; lead_m = int(meta_tr["lead"]) 
    if seq_len and seq_len != seq_len_m:
        print(f"[WARN] seq_len fornecido ({seq_len}) difere do meta ({seq_len_m}). Usando meta.")
    print("📦 Dataset Parquet carregado para treinamento.")

    histories = {}
    models = {}

    early_stopping = TFEarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True, verbose=0)
    reduce_lr = TFReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=6, min_lr=1e-6, verbose=0)

    for name, params in configs.items():
        print(f"\n🚀 Treinando modelo {name}...")
        model = build_lstm_model(seq_len=seq_len_m, x_dim_num=x_dim, y_dim=y_dim, lead=lead_m, params=params, num_countries=4)
        tf.keras.backend.clear_session()
        gc.collect()
        hist = model.fit(
            dataset_train,
            validation_data=dataset_val,
            epochs=200,
            callbacks=[early_stopping, reduce_lr],
            verbose=2,
        )

        histories[name] = hist
        models[name] = model

        print(f"✅ {name} concluído - Val Loss: {min(hist.history['val_loss']):.6f}")

    # ----------------------------
    # Salvando modelos
    # ----------------------------
    for name, model in models.items():
        save_model(model, path = f"./modelos/{problem_name}/{name}")
    
    return models

## Pipelines dos modelos TFT
Pré-processamento em parquet e treino com PyTorch Forecasting (Temporal Fusion Transformer) via Lightning.

In [10]:
# Pipelines dos modelos TFT (imports centralizados na célula 5)

def tft_preproccess_pipeline(
    preproc: TFTPreprocessor,
    destino_dir: str,
    value_cols: List[str],
    save_instance: bool = True,
    train: bool = False
) -> Tuple[TFTPreprocessor, Dict[str, Any]]:

    print("🔄 Carregando dados brutos ...")
    preproc.load_data()

    print("🔤 Encoding ...")
    preproc.encode(encode_cols='datetime', encode_method='time_cycle', train=train)
    preproc.encode(encode_cols='country', encode_method='label', train=train)

    print("✂️ Split train/val/test ...")
    preproc.split_train_val_test(train_size=0.8, val_size=0.1, test_size=0.1, time_col='datetime')

    print("📐 Normalização ...")
    preproc.normalize_splits(value_cols=value_cols, normalization_method='standard', train=train)


    print("🧱 Construindo parquets para TFT ...")
    preproc.build_tft_parquets(
        group_cols=['country'],
        time_col='datetime'
)

    if save_instance and train:
        path = os.path.join(destino_dir, "preprocessor")
        preproc.save_instance(path, name="tft_preproc.pkl")

    print("📦 Parquets TFT salvos. Carregando dataset ...")

    return preproc

def tft_train_pipeline(
    problem_name: str,
    data_dir: str,
    feature_cols: List[str],
    target_cols: List[str],
    seq_len: int,
    lead: int,
    batch_size: int = 128,
    configs: Dict[str, Dict[str, Any]] = None,
):
    """
    Treinamento de modelos TFT (Temporal Fusion Transformer) usando PyTorch Forecasting + Lightning.

    - Consome os parquets gerados por TFTPreprocessor: tft_dataset_train.parquet e tft_dataset_val.parquet
    - Cria TimeSeriesDataSet para treino/validação
    - Constrói o modelo via TemporalFusionTransformer.from_dataset
    - Treina com EarlyStopping e salva checkpoints por preset
    """

    # ----------------------------
    # 1. Carregar dados pré-processados via TFTPreprocessor reutilizando load_tft_dataset
    # ----------------------------
    preproc = TFTPreprocessor(
        data_dir=data_dir,
        model_name='tft_model',
        feature_cols=feature_cols,
        target_cols=target_cols,
        seq_len=seq_len,
        lead=lead,
        country_list=[],
)

    # usa a função para retornar DataFrames — permite aplicar dtypes e criar TimeSeriesDataSet de forma consistente
    df_train = preproc.load_tft_dataset('train', target_col=target_cols[0])
    df_val = preproc.load_tft_dataset('val', target_col=target_cols[0])

    # ----------------------------
    # 2. TimeSeriesDataSet (encoder/decoder feitos internamente)
    # ----------------------------

    train_loader = df_train.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
    val_loader   = df_val.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

    print(f"📦 Dados TFT — batches: train={len(train_loader)} | val={len(val_loader)}")

    # ----------------------------
    # 3. Treinamento por preset
    # ----------------------------
    models = {}
    seed_everything(42)

    accelerator = "gpu" if torch.cuda.is_available() else "cpu"

    for name, params in (configs or {}).items():
        print(f"\n🚀 Treinando TFT preset: {name} [{accelerator}]")

        model = build_tft_model(
            params={
                **params,
                "dataset": df_train,
            }
)

        save_dir = os.path.join("modelos", problem_name, "TFT", name)
        os.makedirs(save_dir, exist_ok=True)

        callbacks = [
            LGEarlyStopping(monitor="val_loss", patience=int(params.get("patience", 5)), mode="min"),
            LGLearningRateMonitor(logging_interval="epoch"),
            LGModelCheckpoint(
                dirpath=save_dir,
                filename="best",
                monitor="val_loss",
                save_top_k=1,
                mode="min",
),
]

        trainer = Trainer(
            max_epochs=int(params.get("epochs", 50)),
            accelerator=accelerator,
            devices=1,
            callbacks=callbacks,
            default_root_dir=save_dir,
            log_every_n_steps=10,
            logger=True,
            precision=32,
)

        trainer.fit(model, train_loader, val_loader)
        print(f"✅ {name} concluído — melhor checkpoint salvo em {save_dir}")

        models[name] = model

    return models

# Capítulo 4: Definição da estrutura dos modelos - Não foi feita otimização de hiperparm
Configuração dos parâmetros dos modelos

In [11]:
# Configurações dos modelos (imports centralizados na célula 5)

gpu_devices = tf.config.list_physical_devices('GPU')
# Configurações dos modelos

configs_linear = {
    "linear": {
        "linear": True,
        "units": [128, 64],
        "dropout": 0.1,
        "lr": 5e-4,
        "l2": 1e-6,
        "act": "leaky-relu",
        "layer_norm": False,
        "mask_value": -999.0,
    }
}

configs_mlp = {
    "mlp": {
        "linear": False,
        "units": [128, 64],
        "dropout": 0.1,
        "lr": 5e-4,
        "l2": 1e-6,
        "act": "relu",
        "layer_norm": False,
        "mask_value": -999.0,
    }
}

configs_lstm = {
    "lstm": {
        "lstm_units": [256, 128],
        "dense_units": [64],
        "dropout": 0 if gpu_devices else 0.15,
        "rec_dropout": 0 if gpu_devices else 0.05,
        "act": "tanh",
        "lr": 1e-4,
        "l2": 1e-6,
        "layer_norm": True,
    }
}

# Presets de TFT compatíveis com build_tft_model (PyTorch Forecasting)
# Campos utilizados: hidden_size, dropout, lstm_layers, num_heads, lr, epochs, patience
config_tft = {
    "tft": {
        "hidden_size": 64,
        "lstm_layers": 1,
        "num_heads": 2,
        "dropout": 0.2,
        "hidden_continuous_size": 32,
        "attention_head_size": 2,
        "learning_rate": 1e-3,
        "patience": 20,
        "epochs": 200,
    }
}

# Capítulo 5: Préprocessamento de dados

In [13]:
# Execução das pipelines(imports centralizados na célula 5)
full_configs = perguntas


## Função de processamento
preprocess_collector = {}
def run_preprocessing(cfg: Dict[str, Any], 
                      preproc_train_linear: Optional[LinearPreprocessor] = None, 
                      preproc_train_lstm: Optional[LSTMPreprocessor] = None,
                      preproc_train_tft: Optional[TFTPreprocessor] = None
                      ):
    """Executa sequencialmente as três pipelines usando a classe unificada Preprocessor.
    Espera que cfg contenha paths e parâmetros comuns.
    """

    destino_dir = cfg["data_dir"]
    lag = cfg.get("lag", 24)
    lead = cfg.get("lead", 24)
    seq_len = cfg.get("seq_len", 24)

    preproc_lin = LinearPreprocessor(
        model_name="Linear",
        lag=lag,
        lead=lead,
        country_list=cfg["countries"],
        feature_cols=cfg["feats"],
        target_cols=cfg["tgts"],
        data_dir=destino_dir,
    )

    preproc_lstm = LSTMPreprocessor(
        model_name="LSTM",
        lag=seq_len,
        lead=lead,
        country_list=cfg["countries"],
        feature_cols=cfg["feats"],
        target_cols=cfg["tgts"],
        data_dir=destino_dir
    )

    preproc_tft = TFTPreprocessor(
        model_name="TFT",
        seq_len=seq_len,
        lead=lead,
        country_list=cfg["countries"],
        feature_cols=cfg["feats"],
        target_cols=cfg["tgts"],
        data_dir=destino_dir
    )

    ## Herdando encoders e normalizadores do preprocessador de treino
    try:
        preproc_lin.encod_objects = preproc_train_linear.encod_objects
        preproc_lin.norm_objects = preproc_train_linear.norm_objects
    except:
        pass

    try:
        preproc_lstm.encod_objects = preproc_train_lstm.encod_objects
        preproc_lstm.norm_objects = preproc_train_lstm.norm_objects
    except:
        pass

    try:
        preproc_tft.encod_objects = preproc_train_tft.encod_objects
        preproc_tft.norm_objects = preproc_train_tft.norm_objects
    except:
        pass



    print("==== PIPELINE LINEAR/MLP ====")
    preproc_lin = linear_preproccess_pipeline(
        preproc=preproc_lin,
        destino_dir=destino_dir,
        seq_len=seq_len,
        masked_value=-999.0,
        value_cols=cfg["vals"],
        save_instance=True,
        train=cfg.get("train", False)
    )

    print("==== PIPELINE LSTM ====")
    preproc_lstm = lstm_preproccess_pipeline(
        preproc=preproc_lstm,
        destino_dir=destino_dir,
        value_cols=cfg["vals"],
        save_instance=True,
        train=cfg.get("train", False)
    )

    print("==== PIPELINE TFT ====")
    preproc_tft = tft_preproccess_pipeline(
        preproc= preproc_tft,
        destino_dir=destino_dir,
        value_cols=cfg["vals"],
        save_instance=True,
        train=cfg.get("train", False)
    )

    return preproc_lin, preproc_lstm, preproc_tft


## Processamento individual de dataset completo, para setting de parâmetros de encodding e decoding
train_preproc_linear, train_preproc_lstm, train_preproc_tft = run_preprocessing(treinamento)

# ## Paralelização da geração de dataset de Perguntas N1A -> N3C
# max_workers = min(len(full_configs), multiprocessing.cpu_count())

# print(f"🚀 Executando {len(full_configs)} tarefas em {max_workers} workers...")

# results = [None] * len(full_configs)
# with ThreadPoolExecutor(max_workers=max_workers) as executor:
#     futures = {executor.submit(run_preprocessing, 
#         item,
#         preproc_train_linear=train_preproc_linear,
#         preproc_train_lstm=train_preproc_lstm,
#         preproc_train_tft=train_preproc_tft): 
#         idx for idx, item in enumerate(full_configs)}
    
#     for i, future in enumerate(as_completed(futures)):
#         idx = futures[future]
#         try:
#             results[idx] = future.result()
#         except Exception as e:
#             print(f"[ERROR] item {idx}: {e}")
#             results[idx] = None
#         if (i + 1) % max(1, len(full_configs)//10) == 0:
#             print(f"  Progresso: {i+1}/{len(full_configs)}")

# print("✅ Tarefa concluída.")


==== PIPELINE LINEAR/MLP ====
🔄 Carregando dados brutos ...
🔤 Encoding de colunas categóricas/temporais ...
✂️ Split train/val/test ...
[DIVIDIDO] train: 83,831 linhas
[DIVIDIDO] val: 10,478 linhas
[DIVIDIDO] test: 10,480 linhas
🔳 Construindo matrizes planas para Linear/MLP ...
[INFO] Ignorando targets não numéricos/bool em build_flat_matrix: ['quantity_MW_lead1', 'quantity_MW_lead2', 'quantity_MW_lead3', 'quantity_MW_lead4', 'quantity_MW_lead5', 'quantity_MW_lead6', 'quantity_MW_lead7', 'quantity_MW_lead8', 'quantity_MW_lead9', 'quantity_MW_lead10', 'quantity_MW_lead11', 'quantity_MW_lead12', 'quantity_MW_lead13', 'quantity_MW_lead14', 'quantity_MW_lead15', 'quantity_MW_lead16', 'quantity_MW_lead17', 'quantity_MW_lead18', 'quantity_MW_lead19', 'quantity_MW_lead20', 'quantity_MW_lead21', 'quantity_MW_lead22', 'quantity_MW_lead23', 'quantity_MW_lead24', 'quantity_MW_lead25', 'quantity_MW_lead26', 'quantity_MW_lead27', 'quantity_MW_lead28', 'quantity_MW_lead29', 'quantity_MW_lead30', 'qu

# Capítulo 6: Treinamento dos modelos
Este capítulo executa, por problema: Linear/MLP (configs_linear), MLP (configs_mlp), LSTM (configs_lstm) e TFT (config_tft), liberando memória entre execuções.

In [ ]:
# Treinamento sequencial dos modelos (imports centralizados na célula 5)


tempo_treino = {}

# Carrega configuração de treinamento
cfg = treinamento

if not cfg:
    print("sem configuração de 'treinamento' configurada na lista de treinamento")
else:
    name = cfg["name"]
    print(f"\n🚀 Iniciando treinamento dos modelos ...")

    print("="*80)
    print("Modelo Linear...")
    # Treinamento Linear
    try:
        tempo_treino["linear"] = {}
        tempo_treino["linear"]["inicio"] = time.time()
        models_linear = linear_train_pipeline(
            problem_name=name,
            data_dir=cfg["data_dir"],
            batch_size=256,
            configs=configs_linear,
        )
        del models_linear
        tempo_treino["linear"]["fim"] = time.time()
        tempo_treino["linear"]["duracao"] = tempo_treino["linear"]["fim"] - tempo_treino["linear"]["inicio"]
    except Exception as e:
        print(f"❌ Erro ao treinar Linear para {name}: {e}")
    finally:
        tf.keras.backend.clear_session()
        gc.collect()

    print("="*80)
    print("Modelo MLP...")
    # Treinamento MLP (configs_mlp)
    try:
        tempo_treino["mlp"] = {}
        tempo_treino["mlp"]["inicio"] = time.time()
        models_mlp = linear_train_pipeline(
            problem_name=name,
            data_dir=cfg["data_dir"],
            batch_size=256,
            configs=configs_mlp,
        )
        del models_mlp
        tempo_treino["mlp"]["fim"] = time.time()
        tempo_treino["mlp"]["duracao"] = tempo_treino["mlp"]["fim"] - tempo_treino["mlp"]["inicio"]
    except Exception as e:
        print(f"❌ Erro ao treinar MLP (configs_mlp) para {name}: {e}")
    finally:
        tf.keras.backend.clear_session()
        gc.collect()

    print("="*80)
    print("Modelo LSTM...")
    # # Treinamento LSTM
    try:
        tempo_treino["lstm"] = {}
        tempo_treino["lstm"]["inicio"] = time.time()
        models_lstm = lstm_train_pipeline(
            problem_name=name,
            data_dir=cfg["data_dir"],
            seq_len=cfg.get("lag") or cfg.get("seq_len"),
            batch_size=256,
            configs=configs_lstm,
        )
        del models_lstm
        tempo_treino["lstm"]["fim"] = time.time()
        tempo_treino["lstm"]["duracao"] = tempo_treino["lstm"]["fim"] - tempo_treino["lstm"]["inicio"]
    except Exception as e:
        print(f"❌ Erro ao treinar LSTM para {name}: {e}")
    finally:
        tf.keras.backend.clear_session()
        gc.collect()


    print("="*80)
    print("Modelo TFT...")
    # # Treinamento TFT (Temporal Fusion Transformer)
    try:
        tempo_treino["tft"] = {}
        tempo_treino["tft"]["inicio"] = time.time()
        models_tft = tft_train_pipeline(
            problem_name=name,
            data_dir=cfg["data_dir"],
            feature_cols=cfg.get("feats") or cfg.get("feature_cols"),
            target_cols=cfg.get("tgts") or cfg.get("target_cols"),
            seq_len=cfg.get("lag"),
            lead=cfg.get("lead"),
            batch_size=256,
            configs=config_tft,
        )
        # libera referência ao retorno (modelos serializados internamente)
        del models_tft
        tempo_treino["tft"]["fim"] = time.time()
        tempo_treino["tft"]["duracao"] = tempo_treino["tft"]["fim"] - tempo_treino["tft"]["inicio"]
    except Exception as e:
        print(f"❌ Erro ao treinar TFT para {name}: {e}")

    finally:
        tf.keras.backend.clear_session()
        gc.collect()

    print(f"✅ Problema {name} concluído — memória limpa\n{'-'*60}")
    for modelo, tempo in tempo_treino.items():
        duracao = tempo.get("duracao", 0)
        print(f"⏱️  Tempo de treino {modelo}: {duracao:.2f} segundos")


🚀 Iniciando treinamento dos modelos ...
Modelo LSTM...
📦 Dataset Parquet carregado para treinamento.

🚀 Treinando modelo lstm...
Epoch 1/200
242/242 - 22s - 91ms/step - loss: 0.2115 - mae: 0.2946 - val_loss: 0.3111 - val_mae: 0.4534 - learning_rate: 1.0000e-04
Epoch 2/200
242/242 - 14s - 57ms/step - loss: 0.0955 - mae: 0.2258 - val_loss: 0.2448 - val_mae: 0.4063 - learning_rate: 1.0000e-04
Epoch 3/200
